# CryptoMarketTracker KPI Analysis (Beyond the Dashboard)

Tableau is great for showing charts, trends, moving averages, and daily volatility,
but it doesn't really answer *how well an asset actually performs* when you look at
risk, consistency, and long-term downside.

So this notebook is where I compute the metrics that investors actually care about:

- **Is the asset far from its All Time High or already near it?**
- **Does it pay off compared to the risk it takes (Sharpe Ratio)?**
- **How bad has its worst crash ever been?**
- **How often does it close positive?**
- **Is it in a low or high volatility environment right now?**

The goal isn’t to generate 20 metrics just to say I did it,
it’s to measure real performance + risk in a way that the dashboard can’t.
All data is pulled straight from the SQLite database I built earlier.
I will be looking at Bitcoin (BTC) in this analysis, but the same code can be applied to any asset.

## Load CSV Data Into SQLite Database

This cell reads all raw crypto CSV files, extracts the important columns, assigns each asset a `coin` and `symbol` based on its filename, and saves everything into an SQLite database for analysis.

Details:
- Automatically detects timestamp and price columns
- Standardizes timestamps
- Labels assets using file names (ex: BTC, ETH)
- Skips duplicates when rerunning the script
- Stores all data in `CryptoData.db` under the `crypto_prices` table

In [1]:
import sqlite3
import pandas as pd
import glob
import os
from pathlib import Path
from datetime import datetime

# File paths
DatabasePath = "CryptoData.db"
RawDataPath = "data/raw/*.csv"

# Create database connection
connect = sqlite3.connect(DatabasePath)
cursor = connect.cursor()

# Create SQL table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS crypto_prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    coin TEXT,
    symbol TEXT,
    timestamp TEXT UNIQUE,
    price REAL,
    market_cap REAL,
    volume REAL
);
""")

# Grab all CSV files in data/raw
files = glob.glob(RawDataPath)

# Loop through each CSV
for file in files:
    print(f"Loading {file}...")

    df = pd.read_csv(file)

    # Make column names lowercase for easier matching
    df.columns = df.columns.str.lower()

    # Detect and format timestamp column safely
    time_cols = [col for col in df.columns if "time" in col or "date" in col]
    if time_cols:
        df["timestamp"] = pd.to_datetime(df[time_cols[0]], errors="coerce").astype(str)
    else:
        print("No timestamp column found, skipping file.")
        continue

    # Detect price column safely
    price_cols = [col for col in df.columns if "price" in col or "close" in col]
    if price_cols:
        df["price"] = pd.to_numeric(df[price_cols[0]], errors="coerce")
    else:
        print("No price column found, skipping file.")
        continue

    # Optional columns (market cap & volume)
    df["market_cap"] = pd.to_numeric(df.get("market_cap", 0), errors="coerce")
    df["volume"] = pd.to_numeric(df.get("volume", 0), errors="coerce")

    # Tag coin & symbol based on filename
    name = os.path.basename(file).lower()
    if "bitcoin" in name or "btc" in name:
        df["coin"], df["symbol"] = "Bitcoin", "BTC"
    elif "ethereum" in name or "eth" in name:
        df["coin"], df["symbol"] = "Ethereum", "ETH"
    else:
        df["coin"] = name.split(".")[0]
        df["symbol"] = None

    # Insert into SQL database using insert-ignore to avoid duplicates
    for _, row in df.iterrows():
        cursor.execute("""
        INSERT OR IGNORE INTO crypto_prices (coin, symbol, timestamp, price, market_cap, volume)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (row.coin, row.symbol, row.timestamp, row.price, row.market_cap, row.volume))

# Commit and close
connect.commit()
connect.close()

print("All data loaded into SQLite database")

Loading data/raw/ethereumPrices.csv...
Loading data/raw/bitcoinPrices.csv...
All data loaded into SQLite database


## Core KPIs

These three are the backbone of performance and risk analysis:

### 1. % From ATH (All Time High)
Tells us how far the asset is from its highest recorded price.
- If it’s close to 0%, it’s near the top
- If it’s -40% or lower, it’s way off the highs and has more recovery potential

### 2. Sharpe Ratio (Annualized)
This measures **risk-adjusted return**
- Higher = better return per unit of risk
- Anything above 1.0 is decent, 1.5+ is strong, 2+ is impressive

### 3. Max Drawdown
This is the worst peak → bottom drop in history
- BTC dropping 70%? Yeah that's max drawdown
- This tells me the worst pain possible if someone held through the collapse

These 3 together tell the story of *return, risk, and worst case scenario*.

In [5]:
import numpy as np

df["DailyRet"] = df.groupby("coin")["price"].pct_change()

def CoinStats(group):
    group = group.sort_values("date").reset_index(drop=True)
    
    ath = group["price"].max()
    latest = group["price"].iloc[-1]
    pctAth = (latest - ath) / ath * 100

    returns = group["DailyRet"].dropna()
    if len(returns) > 1 and returns.std() != 0:
        sharpe = (returns.mean() / returns.std()) * np.sqrt(252)
    else:
        sharpe = np.nan

    runMax = group["price"].cummax()
    dd = (group["price"] - runMax) / runMax
    maxDd = dd.min() * 100

    return pd.Series({
        "LatestPrice": latest,
        "ATH": ath,
        "PctFromATH": pctAth,
        "Sharpe": sharpe,
        "MaxDrawdown": maxDd
    })

core = df.groupby("coin").apply(CoinStats).reset_index()
core.to_csv("exports/coreKpis.csv", index=False)

core


/var/folders/6l/x4chl4_n3rqfjk60tqzm_vf80000gn/T/ipykernel_3526/1468220212.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  core = df.groupby("coin").apply(CoinStats).reset_index()


,coin,LatestPrice,ATH,PctFromATH,Sharpe,MaxDrawdown
0,Bitcoin,102949.064222,124773.508231,-17.491248,0.742798,-28.11501


## Extra KPIs (Behavior + Consistency + Market Conditions)

This section is less about raw performance and more about understanding behavior.

### 1. 30-Day Correlation vs Bitcoin
- 1.0 → moves exactly with BTC
- 0.0 → completely unrelated
- negative → moves opposite (rare in crypto)

If everything just copies Bitcoin, it’s not really a unique asset performance.

### 2. Win Rate
How often does it close green?
- 55% win rate means 55% of days had positive return
- I calculate overall **and** last 90 days to see if sentiment is changing

### 3. 30-Day Volatility + Regime
Annualized volatility tells me how violently the price swings.
Then I bucket it into:
- **Low** = chill price movement
- **Medium** = normal crypto chaos
- **High** = danger zone

In [6]:
import os
import numpy as np
import pandas as pd

# ensure exports dir
os.makedirs("exports", exist_ok=True)

# ASSUMPTION: `df` exists in the notebook and contains at least:
# 'date' (parseable), 'coin', and a price column named 'price' (if your column is named differently change below)
PRICE_COL = "price"

# ensure date dtype
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# ensure DailyRet exists
if "DailyRet" not in df.columns:
    df["DailyRet"] = df.groupby("coin")[PRICE_COL].pct_change()

# pivot on the actual price column
pivot = df.pivot(index="date", columns="coin", values=PRICE_COL).sort_index()
ret = pivot.pct_change()

# detect BTC column safely (case-insensitive)
btc_candidates = [c for c in pivot.columns if ("btc" in c.lower()) or ("bitcoin" in c.lower())]
btc_col = btc_candidates[0] if btc_candidates else None

# compute 30-day rolling correlation vs BTC (latest per coin)
if btc_col is not None and btc_col in ret.columns:
    corr_vals = []
    for c in ret.columns:
        try:
            series = ret[c].rolling(30).corr(ret[btc_col])
            last_val = series.dropna().iloc[-1] if series.dropna().size > 0 else np.nan
        except Exception:
            last_val = np.nan
        corr_vals.append({"coin": c, "Corr30": last_val})
    corr = pd.DataFrame(corr_vals)
else:
    corr = pd.DataFrame({"coin": pivot.columns.tolist(), "Corr30": np.nan})

# Win rate overall (percent)
winAll = df.groupby("coin")["DailyRet"].apply(lambda x: (x > 0).mean() * 100).reset_index(name="WinRateAll")

# Latest date per coin and DaysAgo
latestDate = df.groupby("coin")["date"].max().rename("MaxDate").reset_index()
df = df.merge(latestDate, on="coin", how="left")
df["DaysAgo"] = (df["MaxDate"] - df["date"]).dt.days

# Win rate last 90 days (percent)
win90 = (
    df[df["DaysAgo"] <= 90]
    .groupby("coin")["DailyRet"]
    .apply(lambda x: (x > 0).mean() * 100)
    .reset_index(name="WinRate90")
)

# Volatility: last available 30-day rolling std (annualized %, using 252 trading days)
def last_30d_ann_vol(series):
    s = series.dropna()
    if s.empty:
        return np.nan
    roll = s.rolling(30).std()
    last = roll.dropna().iloc[-1] if roll.dropna().size > 0 else np.nan
    return (last * np.sqrt(252) * 100) if pd.notna(last) else np.nan

vol = (
    df.groupby("coin")["DailyRet"]
    .apply(last_30d_ann_vol)
    .reset_index(name="Volatility")
)

# Volatility regime percentiles
p25 = vol["Volatility"].quantile(0.25)
p75 = vol["Volatility"].quantile(0.75)

def Regime(v):
    if pd.isna(v):
        return "n/a"
    if v <= p25:
        return "Low"
    if v <= p75:
        return "Medium"
    return "High"

vol["Regime"] = vol["Volatility"].apply(Regime)

# Merge extras
extra = (
    corr
    .merge(winAll, on="coin", how="left")
    .merge(win90, on="coin", how="left")
    .merge(vol, on="coin", how="left")
)

# write CSV
extra.to_csv("exports/extraKpis.csv", index=False)

# result
extra


,coin,Corr30,WinRateAll,WinRate90,Volatility,Regime
0,Bitcoin,1.0,51.092896,51.086957,37.436176,Low


## Key Takeaways & What They Tell Us
This section summarizes what the KPI analysis means in practice, focusing on performance, risk, and market behavior.

These insights are written to be referenced in:

- GitHub README

- LinkedIn posts

- Portfolio case studies


They answer the questions that price alone can’t:

- Which assets are closest to recovery vs still far from highs?

- Who delivers the best return for the risk taken?

- How severe can downturns realistically get?

- Are assets moving independently or just following Bitcoin?

- Has momentum improved recently or slowed down?

- Are we in a low, normal, or high volatility environment?

The goal is to translate raw metrics into real investment context and market understanding.

## Key Takeaways & What the KPIs Reveal

---

### 1. % From All-Time High (Recovery Positioning)
Shows how close each asset is to reclaiming its historical peak.
- Smaller negative values = closer to recovery
- Useful for spotting momentum without needing a price chart

Example insight:
- Bitcoin is only ~17% off ATH, signaling strong upward momentum

---

### 2. Sharpe Ratio (Return vs Risk)
Measures how much return is earned per unit of volatility.
- Higher = better risk-adjusted performance
- Tells whether volatility is *worth it*

Example insight:
- Bitcoin shows strong return relative to volatility, not just price gains

---

### 3. Max Drawdown (Worst-case loss)
Captures the deepest historical crash.
- Helps set expectations on downside risk
- Shows "how bad things can get" during a crash

Example insight:
- Bitcoin’s worst drawdown illustrates true risk tolerance required

---

### 4. Win Rate (Consistency of Positive Days)
Percent of days the asset finished higher than the day before.
- Shows consistency, not just total return
- Helps distinguish trends vs randomness

Example insight:
- Bitcoin stays near a 51% win rate — slow, steady grind upward

---

### 5. 30-Day Correlation to Bitcoin (Market Dependence)
Shows whether assets move independently or just mirror Bitcoin.
- 1.0 = moves exactly with BTC
- Lower = more diversification potential

Example insight:
- Bitcoin correlation = 1.0, meaning recent moves are driven by macro trend

---

### 6. Volatility Regime (Market Environment)
Classifies volatility into: Low, Medium, High
- Helps understand whether we’re in a stable or chaotic market environment

Example insight:
- Current state shows **Low volatility**, reinforcing a calm trend phase

## Final Summary

The KPI analysis goes beyond price charts and measures whether an asset is actually
performing well relative to the risk required to hold it.

Bitcoin is currently:

- ~17% below its all-time high — positioned close to full recovery
- Delivering strong risk-adjusted performance (Sharpe ≈ 0.74)
- Experiencing a shallow historical max drawdown compared to prior cycles
- Posting a consistent win rate over time (~51% of days finish positive)
- Moving in perfect correlation to broader crypto momentum (Corr30 = 1.0)
- Operating in a **Low volatility** regime, signaling a stable environment

In short:

Bitcoin is not just trending up — it is doing so *efficiently and with controlled risk*.

These KPIs quantify narrative:

- Return vs. risk (Sharpe)
- Current strength vs. prior peaks (% from ATH)
- Downside risk (Max Drawdown)
- Momentum and consistency (Win Rate)
- Market dependence (Correlation)
- Market environment (Volatility Regime)

Instead of relying on price movement alone, this analysis proves whether the asset is
worth holding from a performance, risk, and consistency standpoint.